In [21]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
from numpy import random
from osgeo import gdal, ogr
#function for processing russia,ukraine master shapefile
def oblast_select():
    print('Oblast Select Function')
    print('ID Options: ', )
    ukr_rus_dat = pd.read_csv("/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/master/ukr_rus_master.csv")
    rus_shp = gpd.read_file("/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/data/shapefile/Russia_admin_clean/rus_admin_cl.shp")
    ukr_shp = gpd.read_file("/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/data/shapefile/Ukraine_admin/regions.shp")
    ukr_shp['ID'] = ukr_shp['ID'] + 100
    ukr_shp = ukr_shp[['ID','geometry']]
    rus_shp = rus_shp.rename(columns = {"ID_1":"ID"})
    rus_shp = rus_shp[['ID','geometry']]
    shp_stack = [rus_shp,ukr_shp]
    shp = pd.concat(shp_stack)
    #merge and set geometry of master dataset
    x = ukr_rus_dat.merge(shp, on = 'ID').set_geometry('geometry')
    m = x[['ID','Country','Oblast-eng','geometry']].drop_duplicates()
    id_select = int(input('Enter ID of which oblast to select: '))
    #feature collection for ee conversion
    selected_oblast = m[m['ID']== id_select]
    print('Oblast Select Completed')
    print('|----------------------------------------|')
    return selected_oblast

#function for reprojecting/warping original mask
def warp_raster():
    from osgeo import gdal
    print('Warp Raster function')
    print('Set to convert to 4326')
    #enter directory folder
    folder_input = '/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/Russia'
    folder_output = '/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/vectorized_mask/raster_reproj'
    #input filepath name
    print('Example: Russia_2020')
    raster_name = input('Enter name of input raster from folder: ')
    print('Example: Russia_2020_4326')
    output_raster = input('Enter name of output raster: ')
    mask_input = folder_input + '/' + raster_name + '.tif'
    mask_output = folder_output + '/' + output_raster + '.tif'
    #open raster
    input_raster = gdal.Open(mask_input)
    warp = gdal.Warp(mask_output,mask_input,dstSRS='EPSG:4326')
    print('Warp Raster Completed')
    print('|----------------------------------------|')
    return warp
     
#function for clipping winter wheat mask to shapefile
def clip_raster_with_shape():
    print('Clip raster with shape function')
    from osgeo import gdal, ogr
    folder_input = '/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/vectorized_mask/raster_reproj'
    print('Folder Path Input\n', folder_input)
    print('Example: Russia_2020_4326')
    raster_name = input('Enter name of input raster: ')
    input_raster = folder_input + '/' + raster_name + '.tif'
    folder_output = '/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/vectorized_mask/clipped_raster'
    print('Folder output path\n',folder_output)
    output_name = raster_name + '_clipped.tif'
    output_raster = folder_output + '/' + output_name
    shape_clip = '/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/subselect_shapefile/oblast_select.shp'
    OutTile = gdal.Warp(output_raster,input_raster,cutlineDSName = shape_clip)
    OutTile = None
    print('Clip Raster completed')
    print('|----------------------------------------|')
    return OutTile

#function for polygonizing clipped winter wheat mask
#function to polygonize mask, returns a shapefile projected to 4326
def polygonize_gdal():
    from osgeo import gdal
    print('Polygonize raster with GDAL function')
    folder_input = '/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/vectorized_mask/clipped_raster'
    print('Folder Path Input\n',folder_input)
    print('Example: Russia_2020_4326_clipped')
    raster_name = input('Enter name of input raster: ')
    input_raster = folder_input + '/' + raster_name + '.tif'
    input_raster = gdal.Open(input_raster)
    folder_output = '/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/vectorized_mask/polygonized/'
    folder_output_name = raster_name + '_poly.shp'
    output_shp = folder_output + folder_output_name
    drv = ogr.GetDriverByName('ESRI Shapefile')
    outfile = drv.CreateDataSource(output_shp)
    band = input_raster.GetRasterBand(1)
    outlayer = outfile.CreateLayer('polygonized raster', srs = None )
    newField = ogr.FieldDefn('DN', ogr.OFTReal)
    outlayer.CreateField(newField)
    gdal.Polygonize(band, None, outlayer, 0, [])
    outfile = None
    shapefile = gpd.read_file(output_shp)
    shapefile = shapefile.set_crs('EPSG:4326')
    shapefile.to_file(output_shp)
    print('Polygonize Raster Completed')
    print('|----------------------------------------|')
    return shapefile



#define function for filtering out unneeded DN,calc area, and divide by 1000
#polygonized mask, year of mask (TO DO: code in grabbing year of mask)
def mask_poly_calc(poly_mask):
    print('Calculate polygon mask function')
    year = int(input('Enter year of mask: '))
    poly_mask = poly_mask.to_crs("EPSG:3857")
    poly_mask = poly_mask[poly_mask['DN'] != 0]
    poly_mask['m2/1000'] = poly_mask.area/1000
    poly_mask = poly_mask.drop(columns = ['DN'])
    poly_mask = poly_mask[poly_mask['m2/1000'] > 6000]
    #create column for storing the year that the mask represents
    poly_mask['year'] = year
    poly_mask = poly_mask.to_crs('EPSG:4326')
    print('Calculate Polygon Mask Completed')
    poly_mask.to_file('/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/vectorized_mask/poly_filt/mask_filt_'+ str(year)) 
    print('|----------------------------------------|')
    return poly_mask

##function for doing getting only intersecting polygons

#function for overlaying two masks over one another and getting areas NOT intersected
##additonally reorganizes columns
##explodes polygons into individual units
##recalculates exploded polygons
def overlay_mask(mask1,mask2):
    print('Overlay Mask: Symmetric Differences')
    #conduct overlay
    mask = gpd.overlay(mask1,mask2, how = 'symmetric_difference')
    mask = mask.to_crs('EPSG:3857')
    #fillna in columns
    mask.iloc[:, [0,1,2,3]] = mask.iloc[:, [0,1,2,3]].fillna(0)
    #combine columns
    mask['year'] = mask['year_1'].map(int) + mask['year_2'].map(int)
    mask['m2/1000'] = mask['m2/1000_1'].map(float) + mask['m2/1000_2'].map(float)
    #drop unneeded columns
    mask = mask.iloc[:,[4,5,6]]
    #explode geometries
    mask = mask.explode()
    #recalculate area
    mask['m2/1000'] = mask.area/1000
    #filter out small areas
    mask = mask[mask['m2/1000'] > 250]
    #reindex
    mask = mask.reset_index()
    #drop columns
    mask = mask.iloc[:,[2,3,4]]
    mask = mask.to_crs('EPSG:4326')
    print('Overlay Mask Completed')
    print('|----------------------------------------|')
    return mask

#select random points in each polygon at n samples and return point with associated year
def random_points(mask):
    print('Random Points in Polygon')
    import time
    #userinput
    samples = int(input('Enter number of samples: '))
    #dissolve all polygons by year and reset index
    mask = mask_final.dissolve(by = 'year').reset_index()
    #initalize point and year ls
    points = []
    year_ls = []
    #run through each row
    for index, row in mask.iterrows():
            start_time = time.time()
            #grab year of polygon
            year = row[0]
            #initalize counter
            i = 0
            sub_mask = mask[mask['year'] == year]
            min_x, min_y, max_x, max_y = sub_mask.bounds.iloc[0]
            print('Subset Mask Year: ',sub_mask.year)
            while i < samples:
                point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
                if sub_mask.contains(point).bool():
                    #append points to point list
                    points.append(point)
                    #append year to year list
                    year_ls.append(year)
                    #add to counter
                    i += 1
                    print('Sample: ', i)
                    #add points,year to a dataframe
                    point_df = pd.DataFrame(list(zip(points,year_ls)))
                    print(point_df)
                    print("Execution Time:", "%s seconds" % (time.time()-start_time))
                    print('|--------------------------------------------------------|')
                    
    #convert points from list to geodataframe and assign geometry and crs
    point_df = point_df.rename(columns = {0:'geometry',1:'year'})
    point_gdf = gpd.GeoDataFrame(point_df,geometry = 'geometry',crs = "EPSG:4326")
    #idk why but it fixes a bug so just go with it
    #x = pd.DataFrame(point_gdf)
    #point_gdf = gpd.GeoDataFrame(x)
    return point_gdf

#function to grab pixel values from each point

#function to compute statistics from each inidivudal pixel value

In [ ]:
oblast_select = oblast_select()
oblast_select.to_file('/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/subselect_shapefile/oblast_select.shp')

In [9]:
#2020
warp_raster()
clip_raster_with_shape()
mask_2020 = polygonize_gdal()
mask_poly_2020 = mask_poly_calc(mask_2020)
#2019
warp_raster()
clip_raster_with_shape()
mask_2019 = polygonize_gdal()
mask_poly_2019 = mask_poly_calc(mask_2019)
#2018
warp_raster()
clip_raster_with_shape()
mask_2018 = polygonize_gdal()
mask_poly_2018 = mask_poly_calc(mask_2018)
#2017
warp_raster()
clip_raster_with_shape()
mask_2017 = polygonize_gdal()
mask_poly_2017 = mask_poly_calc(mask_2017)
#2016
warp_raster()
clip_raster_with_shape()
mask_2016 = polygonize_gdal()
mask_poly_2016 = mask_poly_calc(mask_2016)

Warp Raster function
Set to convert to 4326
Example: Russia_2020
Enter name of input raster from folder: Russia_2020
Example: Russia_2020_4326
Enter name of output raster: Russia_2020_4326
Warp Raster Completed
|----------------------------------------|
Clip raster with shape function
Folder Path Input
 /Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/vectorized_mask/raster_reproj
Example: Russia_2020_4326
Enter name of input raster: Russia_2020_4326
Folder output path
 /Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/vectorized_mask/clipped_raster
Clip Raster completed
|----------------------------------------|
Polygonize raster with GDAL function
Folder Path Input
 /Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/vectorized_mask/clipped_raster
Example: Russia_2020_4326_clipped
Enter name of input raster: Russia_2020_4326_clipped
Polygonize Raster Completed
|----------------------------------------|
Calculate p

In [11]:
mask_20_19 = overlay_mask(mask_poly_2020,mask_poly_2019)
mask_18_17 = overlay_mask(mask_poly_2018,mask_poly_2017)
mask_20_17 = overlay_mask(mask_20_19,mask_18_17)
mask_final = overlay_mask(mask_20_17,mask_poly_2016)
mask_final.to_file('/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/vectorized_mask/test/mask_final.shp')

|----------------------------------------|
|----------------------------------------|
|----------------------------------------|
|----------------------------------------|


In [22]:
random_points = random_points(mask_final)
random_points.to_file('/Users/christianabys/Desktop/School/Maryland/Research/Wheat_Forecast/Mask/vectorized_mask/test/test_points.shp')

Enter number of samples: 1
Subset Mask Year:  0    2016
Name: year, dtype: int64
Sample:  1
                                             0     1
0  POINT (45.25281916449994 48.91774618170561)  2016
Execution Time: 0.9658081531524658 seconds
|--------------------------------------------------------|
Subset Mask Year:  1    2017
Name: year, dtype: int64
Sample:  1
                                             0     1
0  POINT (45.25281916449994 48.91774618170561)  2016
1   POINT (43.15739790413405 50.0556940846176)  2017
Execution Time: 10.08863615989685 seconds
|--------------------------------------------------------|
Subset Mask Year:  2    2018
Name: year, dtype: int64
Sample:  1
                                             0     1
0  POINT (45.25281916449994 48.91774618170561)  2016
1   POINT (43.15739790413405 50.0556940846176)  2017
2  POINT (41.99139161196615 51.15377525776255)  2018
Execution Time: 0.04572606086730957 seconds
|-----------------------------------------------------